In [1]:
import requests
import gradio as gr
import json
from IPython.display import Markdown, display

E:\Anaconda\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
import requests

system_message = "You are a helpful assistant"

def message_gpt(prompt):
    response = requests.post(
        "http://localhost:11434/api/chat",
        json={    
            "model": "llama3.2",
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ]
        },
        stream=False
    )

    full_reply = ""

    for line in response.text.strip().split("\n"):
        try:
            data = json.loads(line)
            if "message" in data and "content" in data["message"]:
                full_reply += data["message"]["content"]
        except json.JSONDecodeError:
            continue

    return full_reply if full_reply else "⚠️ No valid message content found."

      

In [3]:
message_gpt("Tell me a joke!")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nI hope that made you laugh! Do you want to hear another one?"

In [4]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [5]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [6]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox",flagging_mode="never").launch(inbrowser=True, auth=("Faiz","123"))

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [7]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [ ]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"


input_message = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
output_message = gr.Markdown(label="Response:")

view = gr.Interface(
    fn= message_gpt,
    title="GPT", 
    inputs=[input_message], 
    outputs=[output_message], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch(inbrowser=True)

In [ ]:
def stream_gpt(prompt):
    stream = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": "llama3.2",
            "stream": True,
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ]
        },
        stream=True
    )

    response = ""
    for chunk in stream.iter_lines():
        if chunk:
            data = json.loads(chunk.decode("utf-8"))
            if "message" in data and "content" in data["message"]:
                response += data["message"]["content"]
                yield response

In [ ]:
stream_gpt("Tell me a joke!")

In [ ]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"


input_message = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
output_message = gr.Markdown(label="Response:")

view = gr.Interface(
    fn= stream_gpt,
    title="GPT", 
    inputs=[input_message], 
    outputs=[output_message], 
    flagging_mode="never"
    )
view.launch(inbrowser=True)